In [1]:
import os
import numpy as np
from pydub import AudioSegment
import wave
import struct
def load_mp3(filename,num):
    '''
    输入mp3文件的名，返回特征数据
    '''
    MP3_File = AudioSegment.from_mp3(file=filename)
    MP3_File.export('shiyan'+str(num)+'.wav', format="wav")#生成wave文件
    #wav文件读取
    '''
    nchannels:声道数,
    sampwidth：采样宽度,
    framerate：帧速率,
    nframes：帧的总量，
    '''
    f = wave.open('shiyan'+str(num)+'.wav','rb')
    params = f.getparams()
    nchannels, sampwidth, framerate, nframes = params[:4]
    strData = f.readframes(nframes)#读取音频，字符串格式
    waveData = np.frombuffer(strData,dtype=np.int16)#将字符串转化为int
    waveData = waveData*1.0/(max(abs(waveData)))#wave幅值归一化
    temp_result = np.reshape(waveData,[nframes,nchannels])
    f.close()
    nframes=5000000
    result=temp_result[:nframes]
    outData = result#待写入wav的数据，这里仍然取waveData数据
    outData=np.expand_dims(outData,axis=0)
    outData = tf.cast(outData, dtype=tf.float32)
    return outData

def make_songs(result):
    test=np.array(result)
    nframes=5000000
    nchannels=2
    waveData=test[:nframes]
    outData = waveData
    outData = np.reshape(outData,[nframes*nchannels,1])
    outfile = 'outy.wav'
    outwave = wave.open(outfile, 'wb')#定义存储路径以及文件名
    sampwidth = 2
    fs = 44100
    data_size = len(outData)
    framerate = int(fs)
    nframes = data_size
    comptype = "NONE"
    compname = "not compressed"
    outwave.setparams((nchannels, sampwidth, framerate, nframes,
        comptype, compname))
    outData=outData
    for i in range(len(outData)):
        v=outData[i]
        outwave.writeframes(struct.pack('h', int(v * 64000 / 2)))#outData:16位，-32767~32767，注意不要溢出
    outwave.close()


In [2]:
import tensorflow as tf
class DCSGAN(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.dense1=tf.keras.layers.Dense(20, activation='relu')
        self.conv1=tf.keras.layers.Conv1D(filters=10,kernel_size=200,padding='same')
        self.dense2=tf.keras.layers.Dense(200000,activation='tanh')
        self.trconv1=tf.keras.layers.Conv1DTranspose(filters=10,kernel_size=20)
        self.conv2=tf.keras.layers.Conv1D(filters=50,kernel_size=2)
        self.reshape1=tf.keras.layers.Reshape(target_shape=(10,5000000))
        self.reshape2=tf.keras.layers.Reshape(target_shape=(200000,10))
        self.reshape3=tf.keras.layers.Reshape(target_shape=(5000000, 2))
        self.flat1=tf.keras.layers.Flatten()
    # def generate(self,inputs):

    def call(self,inputs):
        x=self.conv1(inputs)
        x=self.reshape1(x)
        x=self.dense1(x)
        x=self.dense2(x)
        x=self.reshape2(x)
        x=self.trconv1(x)
        x=self.conv2(x)
        x=self.flat1(x)
        x=x[0][:10000000]
        x=tf.reshape(x,(1,10000000))
        return x
class classification(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.conv1=tf.keras.layers.Conv1D(filters=10,kernel_size=200,padding='same')
        self.pool1=tf.keras.layers.MaxPool1D(pool_size=100)
        self.dense1=tf.keras.layers.Dense(200,activation='tanh')
        self.dense2=tf.keras.layers.Dense(10,activation='relu')
        self.flatten=tf.keras.layers.Reshape(target_shape=(500000,))
        self.output1=tf.keras.layers.Dense(units=2,activation='relu')
    def call(self,inputs):
        temp=self.conv1(inputs)
        temp=self.pool1(temp)
        temp=self.dense1(temp)
        temp=self.dense2(temp)
        temp=self.flatten(temp)
        output=self.output1(temp)
        return output

In [3]:
Decision_model=classification()
checkpoint=tf.train.Checkpoint(model=Decision_model)
checkpoint.restore(tf.train.latest_checkpoint('./save'))
class Decision_Error(tf.keras.losses.Loss):
    def call(self,y_true):
        '''
        y_true是生成网络产生的(1,5000000,2)序列数据
        '''
        temp=Decision_model(y_true)
        loss=tf.square(temp[0][0])
        return loss

In [4]:
train=load_mp3('63892.mp3',1)
temp=np.array(train).flatten()
result=list(temp)
result=np.array(result)
result=tf.cast(result, dtype=tf.float32)
result0=tf.reshape(result,(1,10000000))

In [5]:
train=load_mp3('65766.mp3',3)
temp=np.array(train).flatten()
result=list(temp)
result=np.array(result)
result=tf.cast(result, dtype=tf.float32)
result2=tf.reshape(result,(1,10000000))

In [6]:
train=load_mp3('65919.mp3',4)
temp=np.array(train).flatten()
result=list(temp)
result=np.array(result)
result=tf.cast(result, dtype=tf.float32)
result3=tf.reshape(result,(1,10000000))

In [8]:
with tf.device("/gpu:0"):
    train1=np.random.rand(1,5000000,2)
    train1=tf.cast(train1,dtype=tf.float32)
    model=DCSGAN()
    learning_rate=0.001
    optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate)
    for i in range(1000):
        with tf.GradientTape() as tape:
            test=model(train1)
            loss=tf.reduce_mean(tf.square(result0 - test))+tf.reduce_mean(tf.square(result2 - test))+tf.reduce_mean(tf.square(result3 - test))
        grads=tape.gradient(loss,model.trainable_variables)
        print(loss)
        print(i)
        optimizer.apply_gradients(grads_and_vars=zip(grads,model.trainable_variables))

tf.Tensor(0.06356886, shape=(), dtype=float32)
0
tf.Tensor(0.06352616, shape=(), dtype=float32)
1
tf.Tensor(0.063521855, shape=(), dtype=float32)
2
tf.Tensor(0.063519046, shape=(), dtype=float32)
3
tf.Tensor(0.06351547, shape=(), dtype=float32)
4
tf.Tensor(0.0636632, shape=(), dtype=float32)
5
tf.Tensor(0.06350677, shape=(), dtype=float32)
6
tf.Tensor(0.063502386, shape=(), dtype=float32)
7
tf.Tensor(0.06349751, shape=(), dtype=float32)
8
tf.Tensor(0.06349195, shape=(), dtype=float32)
9
tf.Tensor(0.063490584, shape=(), dtype=float32)
10
tf.Tensor(0.0635727, shape=(), dtype=float32)
11
tf.Tensor(0.0634731, shape=(), dtype=float32)
12
tf.Tensor(0.06346548, shape=(), dtype=float32)
13
tf.Tensor(0.06345739, shape=(), dtype=float32)
14
tf.Tensor(0.06344836, shape=(), dtype=float32)
15
tf.Tensor(0.063438475, shape=(), dtype=float32)
16
tf.Tensor(0.0634276, shape=(), dtype=float32)
17
tf.Tensor(0.06341555, shape=(), dtype=float32)
18
tf.Tensor(0.06340225, shape=(), dtype=float32)
19
tf.Tensor

In [ ]:

# new_test=np.array(test)
# new_test=new_test[0][:10000000]
# new_test=tf.cast(new_test,dtype=tf.float32)
# new_test=tf.reshape(new_test,(1,5000000,2))
# new_test.shape

In [9]:
make_songs(test)